In [1]:
from visualize_data import json_to_dataframe
import pandas as pd

In [ ]:
df = json_to_dataframe("py")

In [ ]:
# 레벨별

def multi_index_to_dict(data):
    temp_dict = dict()
    for (index_1,index_2), value in data.items():
        if index_1 not in temp_dict:
            temp_dict[index_1] = {}
        temp_dict[index_1][index_2]  = value
    return temp_dict


def get_level_dict(df):
    
    # 레벨별 문제 수 : 지금까지 푼 문제 중 레벨별 문제 id의 개수를 의미합니다.
    temp_df = df.copy()
    temp_df["problem_name"] = temp_df["problem_name"].apply(lambda x: re.sub("\(1\)|\s", "", x))
    temp_df = temp_df.groupby("level")["problem_name"].nunique()
    level_pname = temp_df.to_dict()

    # 레벨별 고유한 문제 유형 개수 : 총 13개의 문제 유형 중 레벨별 문제 유형 id의 개수를 의미합니다.
    temp_df = df.copy()
    temp_df["problem_type"] = temp_df["problem_type"].apply(lambda x: re.sub("\(1\)|\s", "", x))
    temp_df = temp_df.groupby("level")["problem_type"].nunique()
    level_ptype = temp_df.to_dict()
    
    # 레벨별 문제 유형 비율 : 사람들이 레벨별 어떤 문제 유형을 많이 풀었는지 살펴보실 수 있습니다.
    temp_df = df.copy()
    temp_df["file_name"] = temp_df["file_name"].apply(lambda x: re.sub("\s", "", x))
    temp_df["problem_type"] = temp_df["problem_type"].apply(lambda x: re.sub("\(1\)|\s", "", x))
    temp_df = temp_df.groupby(["level","problem_type"])["file_name"].nunique()
    level_dcnt_method = multi_index_to_dict(temp_df)
    
    # 레벨별 메소드 이용수, 누적 : 레벨별로 어떤 메소드를 주로 사용하여 문제를 풀이했는지 살펴보실 수 있습니다.
    temp_df = df.groupby(['level','function_method'])['countmethod'].sum()
    level_cnt_method = multi_index_to_dict(temp_df)

    # 메소드 이용 현황, 누적 값 : 문제 풀이 시, 어떤 메소드를 많이 사용했는지 순위 표를 보실 수 있습니다.
    method_cnt = df.groupby("function_method")["countmethod"].sum().sort_values(ascending = False).to_dict()
    
    return level_pname, level_ptype, level_dcnt_method, level_cnt_method, method_cnt


In [1]:
# 연도별

def NaN_to_None(d):
    return {k1 : {k2: None if v2 == -1 else v2 for k2, v2 in v1.items()} \
                    for k1, v1 in d.items()}


def get_year_dict(df):
    
    # 연도별 문제수
    temp_df = df.copy()
    temp_df = temp_df.drop_duplicates('problem_name')
    temp_df.loc[temp_df['company_name']== '월간코드챌린지시즌2','company_name'] = '프로그래머스'
    temp_df.loc[temp_df['company_name']== '월간코드챌린지시즌3','company_name'] = '프로그래머스'
    temp_df = temp_df.groupby('year')['file_name'].count()
    y_cnt = temp_df.to_dict()

    
    # 연도 별 문제 유형 수
    temp_df = df.copy()
    temp_df = temp_df.drop_duplicates('problem_name')
    temp_df.loc[temp_df['company_name']== '월간코드챌린지시즌2','company_name'] = '프로그래머스'
    temp_df.loc[temp_df['company_name']== '월간코드챌린지시즌3','company_name'] = '프로그래머스'
    temp_df = temp_df.sort_values('year', ascending=False)
    
    y_ptype = temp_df.groupby(['year', 'problem_type']).size().unstack().fillna(-1).to_dict()
    y_ptype = NaN_to_None(y_ptype)
    
    return y_cnt, y_ptype

In [ ]:
# 출제업체별

def NaN_to_None(d):
    return {k1 : {k2: None if v2 == -1 else v2 for k2, v2 in v1.items()} \
                    for k1, v1 in d.items()}
    
def get_company_dict(df):

# 회사 별 출제문제 수
    temp_df = df.copy()
    temp_df = temp_df.drop_duplicates('problem_name')
    temp_df.loc[temp_df['company_name']== '월간코드챌린지시즌2','company_name'] = '프로그래머스'
    temp_df.loc[temp_df['company_name']== '월간코드챌린지시즌3','company_name'] = '프로그래머스'

    c_cnt = temp_df.groupby('company_name').count().iloc[:,:1].to_dict()['file_name']
    
# 회사 별 문제 유형

    temp_df = df.copy()
    temp_df = temp_df.drop_duplicates('problem_name')
    temp_df.loc[temp_df['company_name']== '월간코드챌린지시즌2','company_name'] = '프로그래머스'
    temp_df.loc[temp_df['company_name']== '월간코드챌린지시즌3','company_name'] = '프로그래머스'

    c_ptype = temp_df.groupby(['problem_type','company_name'])['file_name'].count().unstack().fillna(-1).to_dict()
    c_ptype = NaN_to_None(c_ptype)
    
    return c_cnt, c_ptype

In [29]:
# 문제유형별

def df_to_dict(df):
    '''
    분석 시각화 데이터를 딕셔너리 자료형으로 변환
    '''
    
    if df.index.nlevels == 1:
        return df.to_dict()

    elif df.index.nlevels == 2:
        temp_dict = {}
        for (first_key, second_key), value in df.items():
            if first_key not in temp_dict:
                temp_dict[first_key] = {}
            temp_dict[first_key][second_key] = value
        return temp_dict

def get_ptype_dict(df):
    '''
    문제유형별 시각화 데이터를 딕셔너리로 변환합니다
    
    Args:
        data: (DataFrame)
    Returns:
        data: (Dictionary)
    '''
    
    # 문제유형별 문제 수
    
    temp_df = df.copy()
    temp_df = temp_df.drop_duplicates('problem_name')
    temp_df = df['problem_type'].value_counts()
    ptype_name = df_to_dict(temp_df)

    # 문제유형별 내장함수,메서드 사용빈도

    temp_df = df.copy()
    temp_df = temp_df.loc[df['function_method'] != 0]
    temp_df = temp_df.loc[df['problem_type'] != '구현'] # 구현(문제 유형 불분명)이 전체 데이터의 80%이므로, 임시 제외
    temp_df = temp_df.groupby(['problem_type', 'function_method'])['countmethod'].sum()
    ptype_function_method = df_to_dict(temp_df)

    # 문제유형별 내장함수 사용빈도

    temp_df = df.copy()
    temp_df = temp_df.loc[df['function_method'] != 0]
    temp_df = temp_df.loc[(df['module'].isna()) & df['function_method'].str.isalpha()] # module 및 메서드( .(dot)으로 시작하는 형태 제외)
    temp_df = temp_df.loc[df['problem_type'] != '구현']
    temp_df = temp_df.groupby(['problem_type', 'function_method'])['countmethod'].sum()
    ptype_function = df_to_dict(temp_df)
    
    # 문제유형별 메서드 사용빈도

    temp_df = df.copy()
    temp_df = temp_df.loc[df['function_method'] != 0]
    temp_df = temp_df.loc[(df['module'].isna()) & df['function_method'].str.startswith('.')]
    temp_df = temp_df.loc[df['problem_type'] != '구현']
    temp_df = temp_df.groupby(['problem_type', 'function_method'])['countmethod'].sum()
    ptype_method = df_to_dict(temp_df)

    return ptype_name, ptype_function_method, ptype_function, ptype_method
    

In [33]:
# 전체 데이터 저장하는 모듈

def get_dict(df):
    '''
    python 문제의 전체 데이터를 dictionary 형태로 변환하여 반환하는 함수입니다.
    '''    

    # 전체 데이터 저장할 딕셔너리
    temp_dict = {}
    
    # 파트별 딕셔너리 데이터 생성

    level_pname, level_ptype, level_dcnt_method, level_cnt_method, method_cnt = get_level_dict(temp_df)  # 레벨별 & 전체적인 메서드 사용 현황
    ptype_name, ptype_function_method, ptype_function, ptype_method = get_ptype_dict(df)  # 문제유형별
    y_cnt, y_ptype = get_year_dict(df) # 연도별
    c_cnt, c_ptype = get_company_dict(df) # 출제업체별

    # 딕셔너리에 파트별 데이터 삽입

    # 레벨별
    temp_dict['level_pname'] = level_pname
    temp_dict['level_ptype'] = level_ptype
    temp_dict['level_dcnt_method'] = level_dcnt_method
    temp_dict['level_cnt_method'] = level_cnt_method
    temp_dict['method_cnt'] = method_cnt

    # 문제유형별
    temp_dict['ptype_name'] = ptype_name
    temp_dict['ptype_function_method'] = ptype_function_method
    temp_dict['ptype_function'] = ptype_function
    temp_dict['ptype_method'] = ptype_method

    # 연도별
    temp_dict['y_cnt'] = y_cnt
    temp_dict['y_ptype'] = y_ptype

    # 출제업체별
    temp_dict['c_cnt'] = c_cnt
    temp_dict['c_ptype'] = c_ptype
    
    return temp_dict

In [46]:
# main.py 에서 실행될 코드

# Standard library import
import json

# Local application library
from visualize_data import json_to_dataframe
from visualize_data import py # 예시
from visualize_data import js # 예시

py_df = json_to_dataframe('py')
js_df = json_to_dataframe('js')

temp_dict = {}
temp_dict['py'] = py.get_dict(py_df) # py.py 모듈의 get_dict
temp_dict['js'] = js.get_dict(js_df) # js.py 모듈의 get_dict

with open("chart_data.json", "w", encoding="utf-8") as f:
    json.dump(temp_dict, f, ensure_ascii=False, indent=4)